# <center> Lesson 4a - Cryptographic Primitives - Symmetric Encryption
## <center> SYSE 549: Secure Vehicle and Industrial Networking
## <center> <img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" /> 
### <center> Instructor: Dr. Jeremy Daily

## Objective
In this lesson, we will introduce a protocol for authentication. This protocol will depend on a pre-shared key and symmetric encryption. Therefore, we'll discuss implementing symmetric encryption recipes and discuss the challenges and issues associated with symmetric encryption algorithms.

## Challenge-Response Authentication

Let's say there is a remote process controller that you would like to interface with. The controller will act like a server and listen for connections. We'll denote the server as S and the user as the client, or C. 


The protocol could follow these steps:
1. The Client requests a seed or nonce.
2. The Server sends a nonce (number used once).
3. The Client combines its unique ID with the nonce, encrypts it with a symmetric key, and sends it back to the Server.
4. The Server decrypts the message, verifies the nonce and checks the client's unique ID against a master list. 

In symbol form, it might look like this:

$ S\rightarrow C: N $

$ C\rightarrow S: C,\{C,N\}_K $

Where $C$ and $S$ represent the client and server when it's used on the left side of the colon. On the right side of the colon $C$ is the unique ID value for the Client, $N$ is a nonce. $K$ is the symmetric encryption key, so everything inside the curly braces is encrypted with K. Commas represent concatenation.

See Secion 4.3.1 in Anderson for an introduction or review for this protocol.
Let's see if we can implement this.

### Number used once
A nonce in the protocol can come from many sources. It depends on the resources and time constraints. Perhaps the best nonce is a universally unique identifier (UUID), which combines randomness and freshness. Another is a real-time clock counter. Very simple nonces are program counters. However, a program counter is easy to reset and get low number of options. The real-time may not be available to an embedded device. We'll use a 32-byte random number as our nonce.

In [1]:
# Use the operating system for random number generation
from os import urandom
number_of_bytes = 28
nonce = urandom(number_of_bytes)
nonce

b'}\xc9r\x95\xe8_\xd0Q\x11\xfb9\xca\xfe\xca\xee\x9c=\xfc\xdc\x9eX\x93w\xb9\xe0\xa1D\x92'

In [2]:
#A little different form
" ".join(["{:02X}".format(b) for b in nonce])

'7D C9 72 95 E8 5F D0 51 11 FB 39 CA FE CA EE 9C 3D FC DC 9E 58 93 77 B9 E0 A1 44 92'

We'll also need an encryption key. Let's use this really bad example:

In [3]:
encryption_key = b'ThisIsNotSecure!'
" ".join(["{:02X}".format(b) for b in encryption_key])

'54 68 69 73 49 73 4E 6F 74 53 65 63 75 72 65 21'

In [4]:
# For an AES-128 cipher, we'll need a 16 byte key.
key_size = len(encryption_key) 
assert key_size == 16

A key derivation function (KDF) may be a better approach for a key based on a password.

### System Users and Their ID
For a proof of concept, a system server may just use names for logins. While this may be what we see as a user, the actual user data is stored in a hash table, or dictionary. Let's make a user dictionary with a fixed length value as the key and different attributes as the values.

In [5]:
# The key is the user number (integer) and the value is 
# the dictionary of all the properties for the user. 
users = {1:{"name":"Jeremy Daily","role":'admin'},
         2:{"name":"Brayan","role":'user'},
         3:{"name":"Fletcher","role":'user'},
         2348597:{"name":"Michaelangelo","role":'guest'}}
users

{1: {'name': 'Jeremy Daily', 'role': 'admin'},
 2: {'name': 'Brayan', 'role': 'user'},
 3: {'name': 'Fletcher', 'role': 'user'},
 2348597: {'name': 'Michaelangelo', 'role': 'guest'}}

In [6]:
# We need the struct libary to convert integers into bytes of the right length
import struct

In [7]:
# For the protocol, we'll also need a client ID, but it needs to be in a 
# fixed width, so we'll pack the user ID in to a fixed width.
client_id = 1
#but this needs to be in bytes:
client_id_bytes = struct.pack(">L",client_id)
" ".join(["{:02X}".format(b) for b in client_id_bytes])

'00 00 00 01'

In [8]:
# The data to encrypt are:
data_to_encrypt = client_id_bytes + nonce
data_to_encrypt

b'\x00\x00\x00\x01}\xc9r\x95\xe8_\xd0Q\x11\xfb9\xca\xfe\xca\xee\x9c=\xfc\xdc\x9eX\x93w\xb9\xe0\xa1D\x92'

In [9]:
len(data_to_encrypt)

32

### Symmetric Encryption
Symmetric encryption is usually implemented as a block cipher with some feedback mode to eliminate patterns. 

We'll start with an example of common implementations available as libaries in Python, starting with the crypto library at
https://cryptography.io/.
This library is maintained and provides access to many features and algorithms needed to work with protocols using modern cryptography.

DISCLAIMER: Currently cryptography.io only supports NIST curves, none of which are considered “safe” by the SafeCurves (https://safecurves.cr.yp.to/) project run by Daniel J. Bernstein and Tanja Lange.

If the libraries don't load, you can install them by typing `pip install --upgrade cryptography` at the command prompt (not Jupyter or Python). This notebook was originally written for version 3.1 of the cryptography library. See the changelog: https://cryptography.io/en/latest/changelog/.

In [10]:
# If you don't have the latest version of the cryptography library installed, 
# you can use it to install in the version that is serving the jupyter notebook
# This method may be necessary if you have multiple versions of Python on your computer. 
#import sys
# Run this if you are connected to the Internet
#!{sys.executable} -m pip install --upgrade --user cryptography
# You may need to restart the Kernel if the requirements are not already up-to-date
%pip install cryptography

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


The following is a brief intro to Symmetric Ciphers where each principal as access to the same secret key. 

https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/?highlight=ecb%20mode#module-cryptography.hazmat.primitives.ciphers

In [11]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

## Advanced Encryption Standard
Unless there is a really compelling reason, you should pick AES as the symmetric cipher. According to Ross Anderson in Section 5.4.2 (third edition):

```The NSA has since 2005 approved AES with 128-bit keys for protecting information up to SECRET and with 192-bit or 256-bit keys for TOPSECRET. So I recommend that you use AES instead of GOST, or Camellia,or even Serpent.  The definitive specification of AES is Federal Information Processing Standard 197, and its inventors have written a book describing its design in detail.``` 

Let's look at how to use AES with cryptography.io.

The ECB mode is not a secure mode, but it is the simplest.
https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/?highlight=ecb%20mode#cryptography.hazmat.primitives.ciphers.modes.ECB

In [22]:
#check to see if the length of the data to encrypt is evenly divided by the key length
len(data_to_encrypt) % key_size == 0

True

In [23]:
# This is an example of a bad mode for the cipher. NEVER USE THIS! You'll see why
cipher = Cipher(algorithms.AES(encryption_key), modes.ECB())
cipher

In [24]:
#Encrypt the data
encryptor = cipher.encryptor()
encrypted_auth_token = encryptor.update(data_to_encrypt) + encryptor.finalize()
encrypted_auth_token

b'\xd5Q\xf7lP9\xd8gw\xfd\xdeB\x12\\\x81\xb3fz\x01\xa1%\xa7Q-\xe2\xf92n\x81\x99O%'

The data in `encrypted_auth_token` gets sent over the network and is decrypted by the server. 

In [25]:
recieved_auth_token = encrypted_auth_token
recieved_auth_token

b'\xd5Q\xf7lP9\xd8gw\xfd\xdeB\x12\\\x81\xb3fz\x01\xa1%\xa7Q-\xe2\xf92n\x81\x99O%'

In [26]:
#The reciever has the same key as the sender
reciever_cipher = Cipher(algorithms.AES(encryption_key), modes.ECB())

In [27]:
# Decrypt the received token
decryptor = reciever_cipher.decryptor()
decrypted_auth_token = decryptor.update(recieved_auth_token) + decryptor.finalize()
decrypted_auth_token

b'\x00\x00\x00\x01}\xc9r\x95\xe8_\xd0Q\x11\xfb9\xca\xfe\xca\xee\x9c=\xfc\xdc\x9eX\x93w\xb9\xe0\xa1D\x92'

In [28]:
received_nonce = decrypted_auth_token[4:]
received_nonce

b'}\xc9r\x95\xe8_\xd0Q\x11\xfb9\xca\xfe\xca\xee\x9c=\xfc\xdc\x9eX\x93w\xb9\xe0\xa1D\x92'

In [29]:
len(received_nonce)

28

In [30]:
# In this model, the nonce was known by the reciever, so it can be validated.
if received_nonce == nonce:
    (clientID,)= struct.unpack(">L",decrypted_auth_token[:4])
    print(f"The authenticated clientID is {clientID}")
else:
    raise Exception("Not Authenticated")


The authenticated clientID is 1


In [31]:
#Change the data in transit:
recieved_auth_token = bytearray(encrypted_auth_token)
recieved_auth_token[6] = 2 #changed the data

# Decrypt the received token
decryptor = reciever_cipher.decryptor()
decrypted_auth_token = decryptor.update(recieved_auth_token) + decryptor.finalize()

#Extract the nonce
received_nonce = decrypted_auth_token[4:]

#Authenticate the clientID
if received_nonce == nonce:
    (clientID,)= struct.unpack(">L",decrypted_auth_token[:4])
    print(f"The authenticated clientID is {clientID}")
else:
    raise Exception("Not Authenticated")

Exception: Not Authenticated

### Section Summary
In this section, we showed a basic challenge response approach using AES encryption and a pre-shared password. This is not something you would want to do in a modern production environment because fixed pre-shared keys are easy to extract from devices. 

# AES Encryption Modes 
### Bitmap image encryption
We want to send a scrambled picture through the Internet. Let's keep the original image file, but encrypt the contents. Here's a blurb about how bmp files are constructed.

https://medium.com/sysf/bits-to-bitmaps-a-simple-walkthrough-of-bmp-image-format-765dc6857393.

Based on this description, we will keep the original 54 bytes in the image file.

By the way, embedding codes and cryptographic information in pictures is something called steganography. This isn't steganography, but you can learn more about it here:

https://www.comptia.org/blog/what-is-steganography

If this notebook is running on a server, we'll likely have to fetch the image from the internet. The CyberTruck Challenge logo is available at

https://raw.githubusercontent.com/SystemsCyber/CyberTruckResources/master/04_Cryptography/cybertruckchallenge.bmp

Let's use the requests library to get the data from the image.

In [32]:
import requests

In [33]:
r = requests.get("https://raw.githubusercontent.com/SystemsCyber/CyberTruckResources/master/04_Cryptography/cybertruckchallenge.bmp")
logo_bytes = r.content

In [35]:
# BMP Images use 54 bytes up front to define the file
offset = 54
#Let's use a graphic as an example 
with open('cybertruckchallenge.bmp','rb') as raw_file:
    logo_bytes = raw_file.read()
data_blocks,extra = divmod(len(logo_bytes)-offset,key_size)
data_blocks

65536

![logo](https://raw.githubusercontent.com/SystemsCyber/CyberTruckResources/master/04_Cryptography/cybertruckchallenge.bmp)

In [36]:
#instantiate another encryptor
encryptor = cipher.encryptor()

#Start the first few bytes the same as the others
encrypted_picture_bytes = logo_bytes[:offset]

#loop through bytes and ECB encrypt them
encrypted_picture_bytes += encryptor.update(logo_bytes[offset:offset+data_blocks*key_size]) + encryptor.finalize()

In [37]:
encrypted_picture_bytes[:120]

b'BM6\x00\x10\x00\x00\x00\x00\x006\x00\x00\x00(\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x01\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\xc4\x0e\x00\x00\xc4\x0e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@<\xb2\xf6\xa3\xdc.\xdc\x8c@oFwj(\xa8@<\xb2\xf6\xa3\xdc.\xdc\x8c@oFwj(\xa8@<\xb2\xf6\xa3\xdc.\xdc\x8c@oFwj(\xa8@<\xb2\xf6\xa3\xdc.\xdc\x8c@oFwj(\xa8@<'

In [40]:
with open('encrypted_logo_ECB.bmp','wb') as out:
    out.write(encrypted_picture_bytes)

OSError: [Errno 9] Bad file descriptor

![encrypted_logo](https://raw.githubusercontent.com/SystemsCyber/CyberTruckResources/master/04_Cryptography/encrypted_logo_ECB.bmp)

The pattern still exists in the picture! This mode of cipher didn't do a good job of eliminating patterns and hiding data. Unfortunatly, this is the default mode in many libaries!

### Bad modes
The ECB mode is nicknamed "electronic coloring book" because it still reveals patterns and is not secure. Other modes should be used instead.

## AES-CBC
Let's use a better mode, the cipher block chaining mode. However, this isn't the best. AES-GCM is considered the best practice.


In [ ]:
### Generate an initialization vector
iv = urandom(key_size)
cipher = Cipher(algorithms.AES(encryption_key), modes.CBC(iv))
encryptor = cipher.encryptor()

#Start the first few bytes the same as the others
encrypted_picture_bytes = logo_bytes[:offset]

#loop through bytes and ECB encrypt them
encrypted_picture_bytes += encryptor.update(logo_bytes[offset:offset+data_blocks*key_size]) + encryptor.finalize()

with open('encrypted_logo_CBC.bmp','wb') as out:
    out.write(encrypted_picture_bytes)

![encrypted_logo](https://raw.githubusercontent.com/SystemsCyber/CyberTruckResources/master/04_Cryptography/encrypted_logo_CBC.bmp)


### No integrity checks
The process of encrypting and decrypting data does nothing to detect data alteration. Therefore, a data integrity check is critical. Let's see this through an example of encrypting the constitution preamble. First, let's encrypt the preamble with the Fernet recipe that includes a timestamp, an acceptable mode, and a message authentication code.

In [ ]:
# Let's start with some plain text:
plain_text = "We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America."
print(plain_text)


We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.


In [ ]:
# Out of the box recipe for using fast symmetric encryption
from cryptography.fernet import Fernet

In [ ]:
# Generate a key suitable for using the Fernet protocol
shared_key = Fernet.generate_key()

In [ ]:
# The key should be kept safe (This is the hard engineering problem)
print(shared_key)

b'4hjbgYqHuYedqVVQXLDYAxqMxdjHsRW2b8bjZo_u3ho='


In [ ]:
len(shared_key)

44

The key generated is base64 encoded using a URL safe library. 

https://tools.ietf.org/html/rfc3548.html decribes the url and filesystem safe encoding.

As described in 
https://www.pythoninformer.com/python-libraries/cryptography/fernet/ or https://www.comparitech.com/blog/information-security/what-is-fernet/,
the generated key is contains two 16-byte randomly generated keys that are concatenated and base64 encoded.

In [ ]:
# We need to see data in a different encoding.
import base64

In [ ]:
#This is actually two 16-byte keys concatenated and  32-byte (256 bit) base-64 random key
key_bytes = base64.urlsafe_b64decode(shared_key)
print(key_bytes)

b'\xe2\x18\xdb\x81\x8a\x87\xb9\x87\x9d\xa9UP\\\xb0\xd8\x03\x1a\x8c\xc5\xd8\xc7\xb1\x15\xb6o\xc6\xe3f\x8f\xee\xde\x1a'


In [ ]:
# There should be 32-bytes of key material
len(key_bytes)

32

In [ ]:
# The first 16 bytes (128 bits) is the signing key
signing_key = key_bytes[:16]
signing_key

b'\xe2\x18\xdb\x81\x8a\x87\xb9\x87\x9d\xa9UP\\\xb0\xd8\x03'

In [ ]:
# The second half of the key is the encryption key
encryption_key = key_bytes[16:]
encryption_key

b'\x1a\x8c\xc5\xd8\xc7\xb1\x15\xb6o\xc6\xe3f\x8f\xee\xde\x1a'

In [ ]:
# create an instance of the Fernet cipher tool
f = Fernet(shared_key)
print(f)

Fernet methods are documented at https://cryptography.io/en/latest/fernet/

In [ ]:
#Modern ciphers only work on numbers, so we need to convert plaintext to bytes
plain_bytes = plain_text.encode('utf-8')
plain_bytes

b'We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.'

In [ ]:
#the result is ready to be sent over the internet or stored on a machine
cipher_token = f.encrypt(plain_bytes)
print(cipher_token) # This is what can be sent across the Internet

b'gAAAAABm0VIUq9BMUaIWcYOR0gjtgL7uF-_D2TMq_m9uHRM2KIZOY2dNRZI6uEWjD9ujwT7rYdNO0ZdAfEYDxp1J5clF-vAcguqxTzaaY4SjIKCFm_YukMjNYB2DsEy0xWnij0WtJg9G97VNWHWcGIlCRXzBJnzT4GnCyR2N7ZbfDNQGYY78F_ZuYyIsBAYH-7K4YzRkDiZLTXHjxOtvjtZuH3eqcD_zuA4tiWmGAHHyws8ONREcZSO0O56l8Er9s3VfriFVNIyCgPynecVApCYxNvc4EAAzp1jFiwIt5g-Xj-RNv-xljX787X4OVy3SF8kDdyhl6Stl8oeWIXXxV1eF9qqMxLKqny-ekddUdjCglFrWkMT_wZnHBKUnWUk8rxft3enInP_utH8YIUlPNkmrlu8yxrGSmnH5-u9N_TGAd1mPdoL5W9Ap8yu8BJ3QcF43foZfiV1DK8WNPRhomnZvBNWvXMgAsBUrnTJ67iPEjKIa2HlH-LOoCVx_gUbab528wzesoHMp'


In [ ]:
#The inverse is to decrypt to get bytes
f.decrypt(cipher_token)

b'We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.'

From https://www.pythoninformer.com/python-libraries/cryptography/fernet/,
The Fernet protocol implements a more complete symmetric cipher system by

* Providing a secure mechanism for generating keys (a key is similar to a password).
* Selection a secure encryption algorithm (AES using CBC mode and PKCS7 padding)
* Randomly allocating a secure initialization vector (IV) or "salt" to make the encryption more secure.
* Timestamping the encrypted message.
* Signing the message (using HMAC and SHA256) to detect any attempts to change it.

Let's disect the cipher token to see these individual attributes.

### Keys from passwords
Let's generate a valid Fernet key with a passcode. The passcode could be shared verbally instead of electronically.
We'll need to use some primitives. Reference: https://cryptography.io/en/latest/fernet/#using-passwords-with-fernet


We also have to use good random numbers for the initialization vectors.
https://docs.python.org/3/library/secrets.html

In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

PBKDF2 = Password Based Key Derivation Function 2

https://cryptography.io/en/latest/hazmat/primitives/key-derivation-functions/#pbkdf2

In [ ]:
# We will start with a desired pass phrase as bytes
passcode = b"I'm a passphrase"
passcode

b"I'm a passphrase"

In [ ]:
# generate 16 random bytes for the initialization vector or salt
iv = urandom(16)
iv

b'\xcb\xef\xe9\x14\xd1\xb1"\xaf\x8c\xa594\xc2\xbd\xa9\xb0'

In [ ]:
#Key derivation function. This is practically  irreversible 
kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=iv,
        iterations=100000,
    )
kdf_result = kdf.derive(passcode)
print(kdf_result)

b'\xdf\xe5\xad[\xdfT\xaa\xc3\x97\xb1>\x95\x88\xec\x1c\xd9\xa7\xaa\x92\n\xd1Mb\xa8\xcaN0\xc4\xa0\xab8\xf6'


In [ ]:
# We should have 32 bytes now
len(kdf_result)

32

In [ ]:
# Make a valid fernet key:
fernet_key = base64.urlsafe_b64encode(kdf_result)
fernet_key

b'3-WtW99UqsOXsT6ViOwc2aeqkgrRTWKoyk4wxKCrOPY='

In [ ]:
# Check to see if you can create a Fernet cipher
f1 = Fernet(fernet_key)
print(f1)

If both parties know the passphrase and salt value, then each party could derive the same key and send encrypted messages.

## Encrypted Data
Let's see how the data is packed and encrypted. 
The enciphered token has the following structure:

1. Version [1 byte] - the only valid value currently is 128.
2. Timestamp [8 bytes] - a 64 bit, unsigned, big-endian integer that indicates when the ciphertext was created. 
3. IV [16 bytes] - the 128 bit Initialization Vector used in AES encryption and decryption.
4. Ciphertext [16 * N Bytes, where N is the number of cipher blocks] - the encrypted version of the plaintext message. This is encrypted using AES128 in CBC mode using the PKCS7 padding algorithm.
5. HMAC [32 bytes] - a 256-bit hash based message authentication code of the concatenated Version, Timestamp, IV, and Ciphertext fields. The HMAC is signed using the signing key section of the Fernet key.

In [ ]:
# Recall the base64 encoded cipher token
cipher_token

b'gAAAAABm0VIUq9BMUaIWcYOR0gjtgL7uF-_D2TMq_m9uHRM2KIZOY2dNRZI6uEWjD9ujwT7rYdNO0ZdAfEYDxp1J5clF-vAcguqxTzaaY4SjIKCFm_YukMjNYB2DsEy0xWnij0WtJg9G97VNWHWcGIlCRXzBJnzT4GnCyR2N7ZbfDNQGYY78F_ZuYyIsBAYH-7K4YzRkDiZLTXHjxOtvjtZuH3eqcD_zuA4tiWmGAHHyws8ONREcZSO0O56l8Er9s3VfriFVNIyCgPynecVApCYxNvc4EAAzp1jFiwIt5g-Xj-RNv-xljX787X4OVy3SF8kDdyhl6Stl8oeWIXXxV1eF9qqMxLKqny-ekddUdjCglFrWkMT_wZnHBKUnWUk8rxft3enInP_utH8YIUlPNkmrlu8yxrGSmnH5-u9N_TGAd1mPdoL5W9Ap8yu8BJ3QcF43foZfiV1DK8WNPRhomnZvBNWvXMgAsBUrnTJ67iPEjKIa2HlH-LOoCVx_gUbab528wzesoHMp'

In [ ]:
cipher_token_bytes = base64.urlsafe_b64decode(cipher_token)
cipher_token_bytes

b'\x80\x00\x00\x00\x00f\xd1R\x14\xab\xd0LQ\xa2\x16q\x83\x91\xd2\x08\xed\x80\xbe\xee\x17\xef\xc3\xd93*\xfeon\x1d\x136(\x86NcgME\x92:\xb8E\xa3\x0f\xdb\xa3\xc1>\xeba\xd3N\xd1\x97@|F\x03\xc6\x9dI\xe5\xc9E\xfa\xf0\x1c\x82\xea\xb1O6\x9ac\x84\xa3 \xa0\x85\x9b\xf6.\x90\xc8\xcd`\x1d\x83\xb0L\xb4\xc5i\xe2\x8fE\xad&\x0fF\xf7\xb5MXu\x9c\x18\x89BE|\xc1&|\xd3\xe0i\xc2\xc9\x1d\x8d\xed\x96\xdf\x0c\xd4\x06a\x8e\xfc\x17\xf6nc",\x04\x06\x07\xfb\xb2\xb8c4d\x0e&KMq\xe3\xc4\xebo\x8e\xd6n\x1fw\xaap?\xf3\xb8\x0e-\x89i\x86\x00q\xf2\xc2\xcf\x0e5\x11\x1ce#\xb4;\x9e\xa5\xf0J\xfd\xb3u_\xae!U4\x8c\x82\x80\xfc\xa7y\xc5@\xa4&16\xf78\x10\x003\xa7X\xc5\x8b\x02-\xe6\x0f\x97\x8f\xe4M\xbf\xece\x8d~\xfc\xed~\x0eW-\xd2\x17\xc9\x03w(e\xe9+e\xf2\x87\x96!u\xf1WW\x85\xf6\xaa\x8c\xc4\xb2\xaa\x9f/\x9e\x91\xd7Tv0\xa0\x94Z\xd6\x90\xc4\xff\xc1\x99\xc7\x04\xa5\'YI<\xaf\x17\xed\xdd\xe9\xc8\x9c\xff\xee\xb4\x7f\x18!IO6I\xab\x96\xef2\xc6\xb1\x92\x9aq\xf9\xfa\xefM\xfd1\x80wY\x8fv\x82\xf9[\xd0)\xf3+\xbc\x04\x9d\xd0p^7~\x86_\x89]C+\xc5\x8d=

In [ ]:
#parse the fields
version = cipher_token_bytes[0]
print("Version: {}".format(version))
time_stamp = cipher_token_bytes[1:9]
print("time_stamp: {}".format(time_stamp))
init_vect = cipher_token_bytes[9:25]
print("init_vect: {}".format(init_vect))
cipher_data = cipher_token_bytes[25:-32]
print("Length of cipher_data: {}".format(len(cipher_data)))
hmac_value = cipher_token_bytes[-32:]
print("hmac_value: {}".format(hmac_value))

Version: 128
time_stamp: b'\x00\x00\x00\x00f\xd1R\x14'
init_vect: b'\xab\xd0LQ\xa2\x16q\x83\x91\xd2\x08\xed\x80\xbe\xee\x17'
Length of cipher_data: 336
hmac_value: b'\x15+\x9d2z\xee#\xc4\x8c\xa2\x1a\xd8yG\xf8\xb3\xa8\t\\\x7f\x81F\xdao\x9d\xbc\xc37\xac\xa0s)'


In [ ]:
#Convert the timestamp bytes into an integer
#This is the number of seconds from the Jan 1, 1970 epoch
import struct
time_integer = struct.unpack(">Q",time_stamp)[0]
time_integer

1724994068

In [ ]:
#convert the timestamp into iso format so we can read it.
from datetime import datetime
datetime.fromtimestamp(time_integer).isoformat()

'2024-08-29T23:01:08'

For more on time operations in Python: https://docs.python.org/3/library/datetime.html

In [ ]:
#Recall (no time constraints)
f.decrypt(cipher_token)

b'We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.'

In [ ]:
# If more than ttl seconds have passed from the encryption, then the token is bad. 
# This is helpful to guarantee the freshness of data coming from a real-time producer.
# (i.e. instrument data from a test vehicle streaming in real-time. )
f.decrypt(cipher_token,ttl = 10)

InvalidToken: 

### AES-CBC decryption
We have the keys used for the symmetric encryption along with the IV, so we can create an AES cipher based on these inputs and decrypt the cipher text. We'll need to import these primitives. Reference: https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

In [ ]:
#Recall
encryption_key

b'\x1a\x8c\xc5\xd8\xc7\xb1\x15\xb6o\xc6\xe3f\x8f\xee\xde\x1a'

In [ ]:
cipher = Cipher(algorithms.AES(encryption_key), modes.CBC(init_vect))
cipher

In [ ]:
decryptor = cipher.decryptor()
decryptor

In [ ]:
#Notice the padding at the end
# if this gives an error, reinitialize the decryptor
clear = decryptor.update(cipher_data) + decryptor.finalize()
clear

b'We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.\t\t\t\t\t\t\t\t\t'

In [ ]:
# The padding is performed by inserting N bytes at the end of the clear bytes
# The pad length N is encoded as the pad value. The pad value is determined as chr(N).
# The inverse of chr(N) is ord(char)
ord('\t')

9

There were 9 b'/t' characters padded to the end of the preamble to make the block divisible by 16.

In [ ]:
from cryptography.hazmat.primitives import padding

In [ ]:
# Let's use the primitive (16 bytes = 128 bits)
unpadder = padding.PKCS7(128).unpadder()
unpadded_clear = unpadder.update(clear)
unpadded_clear

b'We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of A'

In [ ]:
#We are still missing the last part:
unpadder.finalize()

b'merica.'

In [ ]:
#Putting the two together
unpadder = padding.PKCS7(128).unpadder()
unpadded_clear = unpadder.update(clear) + unpadder.finalize()
unpadded_clear

b'We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.'

### Tamper Detection
What if someone manipulates a bit in the cipher_data? The decryption will still work, but the message will be changed. We need to detect these manipulations before presenting actual data. The decryptor does not know if the data bits were rearranged.

Key Point: encryption does not mean authentication. 

In [ ]:
#Change to a bytearray to manipulate
altered_cipher = bytearray(cipher_data)
altered_cipher == bytearray(cipher_data)

True

In [ ]:
#Change a byte
altered_cipher[45] = 0
altered_cipher == bytearray(cipher_data)

False

In [ ]:
# use the same decryptor and decrypt
decryptor = cipher.decryptor()
decryptor.update(cipher_data) + decryptor.finalize()

b'We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.\t\t\t\t\t\t\t\t\t'

In [ ]:
# in cipher block chaining (CBC) mode, some of the original text will come through after manipulating a byte.
decryptor = cipher.decryptor()
decryptor.update(altered_cipher) + decryptor.finalize()

b'We the People of the United Stat\xab\xbd\xc8f\x94\xb7\re\xfd\x10a\xf9k\x07\xeb\xb3form a more p\x95rfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.\t\t\t\t\t\t\t\t\t'

How can someone tell if the enciphered data was manipulated?

Ans: use a message authentication code.

https://cryptography.io/en/latest/hazmat/primitives/mac/hmac/

In [ ]:
from cryptography.hazmat.primitives import hashes, hmac

In [ ]:
#recall
signing_key

b'\xe2\x18\xdb\x81\x8a\x87\xb9\x87\x9d\xa9UP\\\xb0\xd8\x03'

In [ ]:
#Add the concatenated message bytes to the HMAC and produce the digest
h = hmac.HMAC(signing_key,hashes.SHA256())
h.update(cipher_token_bytes[:-32])
new_hmac_digest = h.finalize()
new_hmac_digest

b'\x15+\x9d2z\xee#\xc4\x8c\xa2\x1a\xd8yG\xf8\xb3\xa8\t\\\x7f\x81F\xdao\x9d\xbc\xc37\xac\xa0s)'

In [ ]:
#Compare
hmac_value == new_hmac_digest

True

In [ ]:
# Using verify
h = hmac.HMAC(signing_key,hashes.SHA256())
h.update(cipher_token_bytes[:-32])
h.verify(cipher_token_bytes[-32:])
# If the verify command doesn't raise an error, it's ok.
print("Message Authenticated")

Message Authenticated


In [ ]:
# Using verify
h = hmac.HMAC(signing_key,hashes.SHA256())
# Remove the first byte
h.update(cipher_token_bytes[1:-32])
h.verify(cipher_token_bytes[-32:])

InvalidSignature: Signature did not match digest.

In [ ]:
# Use a try -except block to catch the verification error
from cryptography import exceptions
h = hmac.HMAC(signing_key,hashes.SHA256())
# Remove the first byte
h.update(cipher_token_bytes[1:-32])
try:
    h.verify(cipher_token_bytes[-32:])
    print("Message Authenticated")
except exceptions.InvalidSignature:
    print("Invalid Signature")

Invalid Signature


## Section Summary
In this section we disected the Fernet protocol for symmetric encryption. In this process we discussed
* Improper Modes of Encryption 
* Key Generation
* Keys from Passwords
* Availability Controls with Timestamps
* Confidentiality Controls using AES in the cipher block chaining mode
* Integrity Controls and Tamper Detection using HMAC
The Fernet recipe is a good starting point to send data across the Internet between trusted parties with a shared key.